In [1]:

import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
st = pd.HDFStore(os.path.expanduser("/Users/nipunbatra/wiki-15min.h5"))


In [3]:
import  os

import numpy as np
import pandas as pd
import pickle
import sys
from subprocess import Popen
sys.path.append("../../code")
from features import feature_map
import subprocess

def _save_results(appliance, lat, feature_comb, test_home, pred_df):
    pred_df.to_csv(os.path.expanduser("~/collab_subset/%s_%d_%s_%d.csv" %(appliance, lat, '_'.join(feature_comb), test_home)))

out_overall = pickle.load(open('../../data/input/all_regions.pkl', 'r'))

region = "Austin"

df = out_overall[region]

df_copy = df.copy()
#drop_rows_having_no_data
o = {}
for h in df.index:
    o[h]=len(df.ix[h][feature_map['Monthly+Static']].dropna())
num_features_ser = pd.Series(o)
drop_rows = num_features_ser[num_features_ser==0].index

df = df.drop(drop_rows)
dfc = df.copy()


df = df.rename(columns={'house_num_rooms':'num_rooms',
                        'num_occupants':'total_occupants',
                        'difference_ratio_min_max':'ratio_difference_min_max'})



In [4]:
df = df[(df.full_agg_available==1)&(df.md_available==1)]


In [5]:
appliance_cols = []
for appliance in ['wm','mw','oven','fridge','hvac','dw']:
    if appliance=="hvac":
        start, stop = 5,11
    else:
        start, stop = 1,13
    appliance_cols.append([appliance+"_"+str(m) for m in range(start, stop)])

In [6]:
all_cols = reduce(lambda x, y: x+y, appliance_cols)


In [7]:
all_homes = df[all_cols].dropna().index

In [146]:
out_df = {}
mat_appliance={}
for appliance_name in ['fridge','hvac','wm','mw','oven','dw','aggregate']:
    print appliance_name
    out_df[appliance_name] = []
    for home in all_homes[:]:
        df_st = st['/%d' %home]['2014']
        cols = df_st.columns
        try:
            if appliance_name=="fridge":
                df = df_st['refrigerator1']['2014-5-1':'2014-10-31'].resample("15T").astype('float64').head(96*200)
            elif appliance_name=="hvac":
                df = df_st['air1']['2014-5-1':'2014-10-31'].resample("15T").astype('float64').head(96*200)
                if 'air2' in cols:
                    df+= df_st['air2']['2014-5-1':'2014-10-31'].resample("15T").astype('float64').head(96*200)
            elif appliance_name=="wm":
                df = df_st['clotheswasher1']['2014-5-1':'2014-10-31'].resample("15T").astype('float64').head(96*200).astype('float64')
            elif appliance_name=="dw":
                df = df_st['dishwasher1']['2014-5-1':'2014-10-31'].resample("15T").astype('float64').head(96*200).astype('float64')
            elif appliance_name=="mw":
                df = df_st['microwave1']['2014-5-1':'2014-10-31'].resample("15T").astype('float64').head(96*200).astype('float64')
            elif appliance_name=="oven":
                df = df_st['oven1']['2014-5-1':'2014-10-31'].resample("15T").astype('float64').head(96*200).astype('float64')
            elif appliance_name=="aggregate":
                df = df_st['use']['2014-5-1':'2014-10-31'].resample("15T").astype('float64').head(96*200).astype('float64')

            
            df = df.groupby(level=0).last()
            df = pd.DataFrame(df)
            df.columns=['power']
            df["day"] = df.index.dayofyear
            df["minute"] = df.index.hour*60 + df.index.minute
            df = df.pivot(index="day", values="power",columns="minute")
            out_df[appliance_name].append(df)
        except Exception, e:
            print e
    mat_appliance[appliance_name] = pd.concat(out_df[appliance_name]).dropna().T.values

fridge
hvac
wm
mw
oven
dw
aggregate


In [8]:
import scipy.io as sio


In [148]:
sio.savemat("/Users/nipunbatra/Desktop/train.mat",mat_appliance)

In [149]:
!ls -lah /Users/nipunbatra/Desktop/train.mat

-rw-r--r--  1 nipunbatra  staff    15M Sep 13 15:52 /Users/nipunbatra/Desktop/train.mat


# Creating testing data

In [9]:
all_homes = df.index

In [10]:
all_homes

Int64Index([  26,   59,   94,  101,  222,  252,  364,  545,  624,  661,
            ...
            9484, 9609, 9647, 9729, 9912, 9922, 9932, 9933, 9971, 9982],
           dtype='int64', length=105)

In [174]:
out_df = {}
mat_appliance={}
for home in all_homes[:]:
    print home
    df_st = st['/%d' %home]['2015']
        
    df = df_st['use'].resample("15T").astype('float64').astype('float64')


    df = df.groupby(level=0).last()
    df = pd.DataFrame(df)
    df.columns=['power']
    df["day"] = df.index.dayofyear
    df["minute"] = df.index.hour*60 + df.index.minute
    df = df.drop_duplicates()
    df = df.pivot(index="day", values="power",columns="minute")
    mat_appliance["home_"+str(home)] = df.T.values
       

26
59
94
101
222
252
364
545
624
661
871
946
1169
1283
1310
1334
1415
1463
1500
1642
1697
1700
1790
1800
1953
2129
2156
2233
2242
2365
2378
2470
2557
2814
2859
2925
2953
2986
3009
3036
3367
3456
3482
3577
3649
3723
3778
3829
3893
3967
4031
4213
4297
4298
4342
4505
4767
4874
4946
4957
4998
5026
5218
5357
5371
5439
5673
5677
5785
5814
5817
6121
6139
6266
6498
6692
6836
6910
7016
7390
7429
7510
7641
7731
7741
7769
7793
7800
7866
8236
8243
8317
8626
9052
9134
9484
9609
9647
9729
9912
9922
9932
9933
9971
9982


In [177]:
sio.savemat("/Users/nipunbatra/Desktop/test.mat",mat_appliance)

In [178]:
!ls -lah /Users/nipunbatra/Desktop/test.mat

-rw-r--r--  1 nipunbatra  staff    28M Sep 13 16:19 /Users/nipunbatra/Desktop/test.mat


Now the results from DDSC

In [179]:
ddsc_mat = sio.loadmat("/Users/nipunbatra/Downloads/result.mat")

In [193]:
appliance_order = ['dw','fridge','hvac','mw','oven','wm']
starts = range(0, 2190, 365)
ends = range(364, 2190, 365)

In [194]:
starts, ends

([0, 365, 730, 1095, 1460, 1825], [364, 729, 1094, 1459, 1824, 2189])

In [182]:
all_homes

Int64Index([  26,   59,   94,  101,  222,  252,  364,  545,  624,  661,
            ...
            9484, 9609, 9647, 9729, 9912, 9922, 9932, 9933, 9971, 9982],
           dtype='int64', length=105)

In [234]:
out = {}
for i, appliance in enumerate(appliance_order):
    
    out[appliance] ={}
    for home in all_homes[:]:
        try:
            df = pd.DataFrame(ddsc_mat['pri%d' %home])


            temp= df[range(starts[i], ends[i])].unstack()
            temp.index = pd.DatetimeIndex(freq='15T', start='2015-1-1', periods=len(temp))
            out[appliance][home] = temp.resample("1M",how="sum").mul(0.000017).mul(15).T.values
        except Exception, e:
            print e


'[2070 2071 2072 2073 2074 2075 2076 2077 2078 2079 2080 2081 2082 2083 2084\n 2085 2086 2087 2088 2089 2090 2091 2092 2093 2094 2095 2096 2097 2098 2099\n 2100 2101 2102 2103 2104 2105 2106 2107 2108 2109 2110 2111 2112 2113 2114\n 2115 2116 2117 2118 2119 2120 2121 2122 2123 2124 2125 2126 2127 2128 2129\n 2130 2131 2132 2133 2134 2135 2136 2137 2138 2139 2140 2141 2142 2143 2144\n 2145 2146 2147 2148 2149 2150 2151 2152 2153 2154 2155 2156 2157 2158 2159\n 2160 2161 2162 2163 2164 2165 2166 2167 2168 2169 2170 2171 2172 2173 2174\n 2175 2176 2177 2178 2179 2180 2181 2182 2183 2184 2185 2186 2187 2188] not in index'
'[2082 2083 2084 2085 2086 2087 2088 2089 2090 2091 2092 2093 2094 2095 2096\n 2097 2098 2099 2100 2101 2102 2103 2104 2105 2106 2107 2108 2109 2110 2111\n 2112 2113 2114 2115 2116 2117 2118 2119 2120 2121 2122 2123 2124 2125 2126\n 2127 2128 2129 2130 2131 2132 2133 2134 2135 2136 2137 2138 2139 2140 2141\n 2142 2143 2144 2145 2146 2147 2148 2149 2150 2151 2152 2153 2154

In [245]:
output_ddsc = {}
for appliance in appliance_order:
    if appliance=="hvac":
        s, e = 4, 10
    else:
        s, e = 0, 12
    output_ddsc[appliance] = pd.DataFrame(out[appliance]).T[range(s, e)]

In [247]:
pickle.dump(output_ddsc, open('../data/ddsc.pkl','w'))

In [12]:
out_df = {}
mat_appliance={}
non_all_feature_homes = [x for x in dfc.index if x not in all_homes]
for ind, home in enumerate(non_all_feature_homes[:]):
    try:
        print home, ind
        df_st = st['/%d' %home]['2015']

        df = df_st['use'].resample("15T").astype('float64').astype('float64')


        df = df.groupby(level=0).last()
        df = pd.DataFrame(df)
        df.columns=['power']
        df["day"] = df.index.dayofyear
        df["minute"] = df.index.hour*60 + df.index.minute
        df = df.drop_duplicates()
        df = df.pivot(index="day", values="power",columns="minute")
        mat_appliance["home_"+str(home)] = df.T.values
    except:
        pass
       

22 0
68 1
77 2
86 3
93 4
114 5
115 6
121 7
130 8
135 9
160 10
171 11
187 12
243 13
267 14
297 15
347 16
370 17
379 18
410 19
434 20
436 21
457 22
470 23
483 24
484 25
491 26
499 27
503 28
507 29
508 30
573 31
575 32
580 33
585 34
645 35
668 36
739 37
744 38
772 39
781 40
861 41
878 42
890 43
898 44
936 45
954 46
974 47
980 48
994 49
1037 50
1069 51
1086 52
1103 53
1105 54
1153 55
1185 56
1192 57
1202 58
1314 59
1331 60
1403 61
1479 62
1507 63
1508 64
1551 65
1577 66
1586 67
1589 68
1597 69
1617 70
1632 71
1681 72
1714 73
1718 74
1791 75
1792 76
1796 77
1801 78
1832 79
1854 80
1889 81
1947 82
1994 83
2004 84
2018 85
2034 86
2072 87
2075 88
2094 89
2158 90
2171 91
2199 92
2207 93
2335 94
2337 95
2361 96
2366 97
2401 98
2449 99
2458 100
2461 101
2465 102
2472 103
2510 104
2520 105
2532 106
2575 107
2638 108
2641 109
2667 110
2742 111
2751 112
2755 113
2769 114
2787 115
2818 116
2829 117
2845 118
2864 119
2907 120
2945 121
2965 122
2974 123
2980 124
2992 125
3039 126
3044 127
3126 128
3134

In [15]:
sio.savemat("/Users/nipunbatra/Desktop/test_larger.mat",mat_appliance)

In [18]:
!ls -lah /Users/nipunbatra/Dropbox/Public/test.mat

-rw-r--r--@ 1 nipunbatra  staff    28M Sep 13 16:19 /Users/nipunbatra/Dropbox/Public/test.mat
